In [1]:
import numpy as np


Converged at iteration 605
Minimum found at x = 4.917855841128087e-05, f(x) = 2.4185306074117642e-09


In [7]:
def gradient_descent(func, grad_func, initial_guess, learning_rate=0.01, max_iter=1000, tolerance=1e-6):

    current_point = initial_guess
    for iteration in range(max_iter):
        # Calculate the gradient at the current point
        gradient = grad_func(current_point)

        # Update the current point in the opposite direction of the gradient
        next_point = current_point - learning_rate * gradient

        # Check if the change is small enough to stop
        if np.linalg.norm(next_point - current_point) < tolerance:
            print(f"Converged at iteration {iteration}")
            break

        current_point = next_point

    return current_point, func(current_point)

In [13]:
def newton_method(func, grad_func, hess_func, initial_guess, max_iter=1000, tolerance=1e-6):
    current_point = initial_guess
    for iteration in range(max_iter):
        # Calculate the gradient and Hessian at the current point
        gradient = grad_func(current_point)
        hessian = hess_func(current_point)

        # If the Hessian is zero stop the process
        if hessian == 0:
            print("Hessian is zero. Stopping the algorithm.")
            break

        # Update the current point
        next_point = current_point - gradient / hessian

        # Check if the change is small enough to stop
        if np.abs(next_point - current_point) < tolerance:
            print(f"Converged at iteration {iteration}")
            break

        current_point = next_point

    return current_point, func(current_point)

In [15]:
def func(x):
    return 3*x**2+2*x+1

def grad_func(x):
    return 6*x+2
def hess_func(x):
    return 6

# Starting point
initial_guess = 10

min_point, min_value = gradient_descent(func, grad_func, initial_guess)
print(f"Minimum found at x = {min_point}, f(x) = {min_value}")

min_point_n, min_value_n = newton_method(func, grad_func, hess_func, initial_guess)
print(f"Minimum found at x = {min_point_n}, f(x) = {min_value_n}")


Converged at iteration 216
Minimum found at x = -0.3333171205784541, f(x) = 0.6666666674552268
Converged at iteration 1
Minimum found at x = -0.3333333333333339, f(x) = 0.6666666666666667


In [6]:
class Particle:
    def __init__(self, dimensions, bounds):
        self.position = np.random.uniform(bounds[0], bounds[1], dimensions)  # Random initial position
        self.velocity = np.random.uniform(-1, 1, dimensions)  # Random initial velocity
        self.best_position = np.copy(self.position)
        self.best_value = float('inf')  # Initial best value (high)

    def evaluate(self, func):
        value = func(self.position)
        if value < self.best_value:
            self.best_value = value
            self.best_position = np.copy(self.position)

def pso(func, dimensions, bounds, num_particles=5, max_iter=10, w=0.5, c1=1.5, c2=1.5):
    particles = [Particle(dimensions, bounds) for _ in range(num_particles)]

    g_best_value = float('inf')
    g_best_position = None

    for iteration in range(max_iter):
        for particle in particles:
            particle.evaluate(func)

            # Update global best if needed
            if particle.best_value < g_best_value:
                g_best_value = particle.best_value
                g_best_position = np.copy(particle.best_position)

        for particle in particles:
            r1, r2 = np.random.rand(2)
            particle.velocity = w * particle.velocity + c1 * r1 * (particle.best_position - particle.position) + c2 * r2 * (g_best_position - particle.position)
            particle.position += particle.velocity

            particle.position = np.clip(particle.position, bounds[0], bounds[1])

        # Print progress
        print(f"Iteration {iteration+1}/{max_iter}, Best Value: {g_best_value}")

    return g_best_position, g_best_value

def func(x):
    return np.sum(3*x**2+2*x+1)  # sum of squares

# Parameters
dimensions = 2
bounds = (-5, 5)  # Search space for each dimension
num_particles = 5
max_iter = 10

# Run PSO
best_position, best_value = pso(func, dimensions, bounds, num_particles, max_iter)

print(f"Best position found: {best_position}")
print(f"Best value found: {best_value}")

Iteration 1/10, Best Value: 39.57695663102824
Iteration 2/10, Best Value: 2.846726824795659
Iteration 3/10, Best Value: 2.846726824795659
Iteration 4/10, Best Value: 2.775297391939693
Iteration 5/10, Best Value: 2.775297391939693
Iteration 6/10, Best Value: 1.7373319061916295
Iteration 7/10, Best Value: 1.56729382979386
Iteration 8/10, Best Value: 1.56729382979386
Iteration 9/10, Best Value: 1.56729382979386
Iteration 10/10, Best Value: 1.56729382979386
Best position found: [-0.60128965 -0.25468065]
Best value found: 1.56729382979386


In [9]:
def simulated_annealing(func, initial_solution, temp, cooling_rate, max_iter):

    current_solution = np.copy(initial_solution)
    current_value = func(current_solution)
    best_solution = np.copy(current_solution)
    best_value = current_value

    for iteration in range(max_iter):

        neighbor_solution = current_solution + np.random.uniform(-1, 1, size=current_solution.shape)
        neighbor_value = func(neighbor_solution)

        # Compute the change in value
        delta_e = neighbor_value - current_value

        # Decide whether to accept the neighbor solution
        if delta_e < 0 or np.random.rand() < np.exp(-delta_e / temp):
            current_solution = np.copy(neighbor_solution)
            current_value = neighbor_value

            # Update the best solution found so far
            if current_value < best_value:
                best_solution = np.copy(current_solution)
                best_value = current_value

        # Cool down the temperature
        temp *= cooling_rate

        print(f"Iteration {iteration+1}/{max_iter}, Current Temperature: {temp:.4f}, Best Value: {best_value}")

    return best_solution, best_value

def func(x):
    return np.sum(3*x**2+2*x+1)  # sum of squares

# Parameters
initial_solution = np.array([10, 10])  # Starting point
temp = 1000  # Initial temperature
cooling_rate = 0.995  # Cooling rate
max_iter = 10  # Maximum number of iterations

best_solution, best_value = simulated_annealing(func, initial_solution, temp, cooling_rate, max_iter)

print(f"Best solution found: {best_solution}")
print(f"Best value found: {best_value}")

Iteration 1/10, Current Temperature: 995.0000, Best Value: 611.0496798332115
Iteration 2/10, Current Temperature: 990.0250, Best Value: 611.0496798332115
Iteration 3/10, Current Temperature: 985.0749, Best Value: 611.0496798332115
Iteration 4/10, Current Temperature: 980.1495, Best Value: 611.0496798332115
Iteration 5/10, Current Temperature: 975.2488, Best Value: 602.1631808754772
Iteration 6/10, Current Temperature: 970.3725, Best Value: 568.9479839125416
Iteration 7/10, Current Temperature: 965.5206, Best Value: 568.9479839125416
Iteration 8/10, Current Temperature: 960.6930, Best Value: 562.8314918542308
Iteration 9/10, Current Temperature: 955.8896, Best Value: 509.17037003136875
Iteration 10/10, Current Temperature: 951.1101, Best Value: 470.0781209116613
Best solution found: [8.37134283 8.63755712]
Best value found: 470.0781209116613
